In [3]:
!pip install pandas folium

In [4]:
import folium
import pandas as pd

In [5]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [6]:
!pip install requests


In [7]:
import requests

# URL of the CSV file
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

# Send a GET request to the URL
response = requests.get(URL)

# Check if the request was successful
if response.status_code == 200:
    # Write the content to a file on your local machine
    with open("spacex_launch_geo.csv", "wb") as file:
        file.write(response.content)
    print("File downloaded successfully!")
else:
    print(f"Failed to download file. Status code: {response.status_code}")


File downloaded successfully!


In [8]:
import pandas as pd

# Load the CSV file into a DataFrame
spacex_df = pd.read_csv("spacex_launch_geo.csv")

# Display the first few rows to check the data
print(spacex_df.head())

   Flight Number        Date Time (UTC) Booster Version  Launch Site  \
0              1  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1              2  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2              3  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3              4  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4              5  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  Payload Mass (kg)  \
0               Dragon Spacecraft Qualification Unit                0.0   
1  Dragon demo flight C1, two CubeSats,  barrel o...                0.0   
2                             Dragon demo flight C2+              525.0   
3                                       SpaceX CRS-1              500.0   
4                                       SpaceX CRS-2              677.0   

       Orbit         Customer        Landing Outcome  class        Lat  \
0        LEO           SpaceX  Failure   (

In [9]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [10]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [11]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [12]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

In [13]:
site_map

In [14]:
for index, row in launch_sites_df.iterrows():
    # Extract the latitude, longitude, and site name for each row
    latitude = row['Lat']
    longitude = row['Long']
    site_name = row['Launch Site']
    
    # Add a Circle around the launch site
    folium.Circle(
        location=[latitude, longitude],  # The center of the circle
        radius=1000,  # Radius of the circle in meters
        color='#000000',  # Border color
        fill=True,  # Fill the circle with color
        fill_color='#ff6666',  # Fill color (light red for example)
        fill_opacity=0.3  # Transparency of the fill
    ).add_to(site_map)
    
    # Add a Marker with a popup for the launch site
    folium.Marker(
        location=[latitude, longitude],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % site_name
        )
    ).add_to(site_map)

In [15]:
site_map

In [16]:
print(launch_sites_df.head())

    Launch Site        Lat        Long
0   CCAFS LC-40  28.562302  -80.577356
1  CCAFS SLC-40  28.563197  -80.576820
2    KSC LC-39A  28.573255  -80.646895
3   VAFB SLC-4E  34.632834 -120.610745


In [17]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long','class']]
launch_sites_df

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


In [18]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

In [19]:
for index, row in launch_sites_df.iterrows():
    # Extract the latitude, longitude, site name, and launch status
    latitude = row['Lat']
    longitude = row['Long']
    site_name = row['Launch Site']
    launch_status = row['class']
    
    # Map the numeric status to a human-readable string (Success or Failure)
    if launch_status == 1:  # Success
        status_label = 'Success'
        marker_color = 'green'
    else:  # Failure (assuming status 0 means failure)
        status_label = 'Failure'
        marker_color = 'red'
    
    # Add a Circle around the launch site
    folium.Circle(
        location=[latitude, longitude],  # The center of the circle
        radius=1000,  # Radius of the circle in meters
        color=marker_color,  # Border color
        fill=True,  # Fill the circle with color
        fill_color=marker_color,  # Fill color
        fill_opacity=0.3  # Transparency of the fill
    ).add_to(site_map)
    
    # Add a Marker with a popup for the launch site
    folium.Marker(
        location=[latitude, longitude],
        popup=f"<b>{site_name}</b><br>Status: {status_label}",  # Show the status in popup
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:{marker_color};"><b>{site_name}</b></div>'
        )
    ).add_to(site_map)
site_map

In [20]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [21]:
marker_cluster = MarkerCluster()

In [22]:
spacex_df

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0
5,CCAFS LC-40,28.562302,-80.577356,0
6,CCAFS LC-40,28.562302,-80.577356,0
7,CCAFS LC-40,28.562302,-80.577356,0
8,CCAFS LC-40,28.562302,-80.577356,0
9,CCAFS LC-40,28.562302,-80.577356,0


In [23]:
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')
spacex_df

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red
5,CCAFS LC-40,28.562302,-80.577356,0,red
6,CCAFS LC-40,28.562302,-80.577356,0,red
7,CCAFS LC-40,28.562302,-80.577356,0,red
8,CCAFS LC-40,28.562302,-80.577356,0,red
9,CCAFS LC-40,28.562302,-80.577356,0,red


In [24]:
site_map.add_child(marker_cluster)


In [25]:
for index, record in spacex_df.iterrows():
    # Extract the latitude, longitude, and class (success/failure) from the DataFrame
    latitude = record['Lat']  # Replace with your actual column name for latitude
    longitude = record['Long']  # Replace with your actual column name for longitude
    launch_class = record['class']  # 1 for success, 0 for failure
    
    # Set marker color based on the class value
    marker_color = 'green' if launch_class == 1 else 'red'
    
    # Create a folium Marker with custom icon color
    marker = folium.Marker(
        location=[latitude, longitude],
        icon=folium.Icon(color='white', icon_color=marker_color),
        popup=f"Launch Site: {record['Launch Site']}<br>Class: {launch_class}",
    )
    
    # Add the marker to the marker cluster
    marker_cluster.add_child(marker)


In [26]:
site_map

In [27]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [28]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [40]:
launch_site_lat = 28.56338  # Przykład: szerokość geograficzna miejsca startowego
launch_site_lon = -80.57682   # Same here, taking the first launch site

# Define the coordinates of the closest coastline (near Kennedy Space Center)
coastline_lat = 28.56381
coastline_lon = -80.56822

# Calculate the distance between the launch site and the coastline
distance_to_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a folium map centered around the launch site
map_center = [launch_site_lat, launch_site_lon]
site_map = folium.Map(location=map_center, zoom_start=10)

# Create and add a folium marker for the launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(site_map)

# Create and add a folium marker for the coastline
folium.Marker(
    [coastline_lat, coastline_lon],
    popup="Closest Coastline",
    icon=folium.Icon(color="green", icon="leaf")
).add_to(site_map)
# Create a PolyLine to connect the launch site to the coastline
coordinates = [(launch_site_lat, launch_site_lon), (coastline_lat, coastline_lon)]
lines = folium.PolyLine(locations=coordinates, weight=2, color='blue', opacity=0.7)

# Add the PolyLine to the map
site_map.add_child(lines)


# Create and add a marker for the calculated distance between the launch site and coastline
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],  # Place it near the coastline
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_to_coastline),
    )
)
distance_marker.add_to(site_map)

In [41]:
distance_to_coastline

0.8415090393662852

In [42]:
site_map

In [44]:
from math import sin, cos, sqrt, atan2, radians


def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance


launch_site_lat = 28.56338  # Przykład: szerokość geograficzna miejsca startowego
launch_site_lon = -80.57682  # Same here, taking the first launch site

# Define the coordinates of the closest coastline (near Kennedy Space Center)
highway_lat = 28.56387
highway_lon = -80.5709

# Calculate the distance between the launch site and the coastline
distance_to_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

# Create a folium map centered around the launch site
map_center = [launch_site_lat, launch_site_lon]
site_map = folium.Map(location=map_center, zoom_start=10)

# Create and add a folium marker for the launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(site_map)

# Create and add a folium marker for the coastline
folium.Marker(
    [highway_lat, highway_lon],
    popup="Closest Coastline",
    icon=folium.Icon(color="green", icon="leaf")
).add_to(site_map)
# Create a PolyLine to connect the launch site to the coastline
coordinates = [(launch_site_lat, launch_site_lon), (highway_lat, highway_lon)]
lines = folium.PolyLine(locations=coordinates, weight=2, color='blue', opacity=0.7)

# Add the PolyLine to the map
site_map.add_child(lines)

# Create and add a marker for the calculated distance between the launch site and coastline
distance_marker = folium.Marker(
    [highway_lat, highway_lon],  # Place it near the coastline
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_to_highway),
    )
)
distance_marker.add_to(site_map)

In [45]:
site_map

In [46]:
from math import sin, cos, sqrt, atan2, radians


def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance


launch_site_lat = 28.56338  # Przykład: szerokość geograficzna miejsca startowego
launch_site_lon = -80.57682  # Same here, taking the first launch site

# Define the coordinates of the closest coastline (near Kennedy Space Center)
railway_lat = 28.56503
railway_lon = -80.58661

# Calculate the distance between the launch site and the coastline
distance_to_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

# Create a folium map centered around the launch site
map_center = [launch_site_lat, launch_site_lon]
site_map = folium.Map(location=map_center, zoom_start=10)

# Create and add a folium marker for the launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(site_map)

# Create and add a folium marker for the coastline
folium.Marker(
    [railway_lat, railway_lon],
    popup="Closest Coastline",
    icon=folium.Icon(color="green", icon="leaf")
).add_to(site_map)
# Create a PolyLine to connect the launch site to the coastline
coordinates = [(launch_site_lat, launch_site_lon), (railway_lat, railway_lon)]
lines = folium.PolyLine(locations=coordinates, weight=2, color='blue', opacity=0.7)

# Add the PolyLine to the map
site_map.add_child(lines)

# Create and add a marker for the calculated distance between the launch site and coastline
distance_marker = folium.Marker(
    [railway_lat, railway_lon],  # Place it near the coastline
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_to_railway),
    )
)
distance_marker.add_to(site_map)
site_map

In [49]:
from math import sin, cos, sqrt, atan2, radians


def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance


launch_site_lat = 28.56338  # Przykład: szerokość geograficzna miejsca startowego
launch_site_lon = -80.57682  # Same here, taking the first launch site

# Define the coordinates of the closest coastline (near Kennedy Space Center)
city_lat = 28.61263
city_lon = -80.80805

# Calculate the distance between the launch site and the coastline
distance_to_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Create a folium map centered around the launch site
map_center = [launch_site_lat, launch_site_lon]
site_map = folium.Map(location=map_center, zoom_start=10)

# Create and add a folium marker for the launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(site_map)

# Create and add a folium marker for the coastline
folium.Marker(
    [city_lat, city_lon],
    popup="Closest Coastline",
    icon=folium.Icon(color="green", icon="leaf")
).add_to(site_map)
# Create a PolyLine to connect the launch site to the coastline
coordinates = [(launch_site_lat, launch_site_lon), (city_lat, city_lon)]
lines = folium.PolyLine(locations=coordinates, weight=2, color='blue', opacity=0.7)

# Add the PolyLine to the map
site_map.add_child(lines)

# Create and add a marker for the calculated distance between the launch site and coastline
distance_marker = folium.Marker(
    [city_lat, city_lon],  # Place it near the coastline
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_to_city),
    )
)
distance_marker.add_to(site_map)

In [50]:
site_map